In [28]:
!pip install folium geopy timezonefinder pytz
!pip install opening_hours_py

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)


In [42]:
import gpxpy
import folium
import requests
import json
import datetime
import math
import os
from geopy.distance import geodesic
from opening_hours.opening_hours import OpeningHours
from timezonefinder import TimezoneFinder
from zoneinfo import ZoneInfo, ZoneInfoNotFoundError

# @title GPX-Routen-Visualisierer mit POI-Ankunftszeitanalyse
# @markdown ---
# @markdown **Abhängigkeiten:**
# @markdown Führen Sie `pip install gpxpy folium requests opening-hours-py timezonefinder geopy` in Ihrem Terminal aus. `pytz` wird nicht mehr benötigt.
# @markdown
# @markdown **Anleitung:**
# @markdown 1. Tragen Sie Ihre Planungsdaten und Filteroptionen im nächsten Block ein.
# @markdown 2. Ersetzen Sie `'PFAD/ZU/IHRER/DATEI.gpx'` ganz unten mit dem tatsächlichen Pfad zu Ihrer GPX-Datei.
# @markdown 3. Führen Sie die Zelle aus. Die Karte wird unten angezeigt **und als teilbare HTML-Datei gespeichert**.
# @markdown ---

# @markdown ### ⚙️ Optionen für die Routenplanung und Anzeige
# @markdown **1. Fahrparameter:**
average_speed_kmh = 23.0 # @param {type:"number"}
# @markdown Geben Sie Ihr Startdatum und Ihre Startzeit in Ihrer **lokalen Zeitzone** an.
start_date = "2025-07-05" # @param {type:"date"}
start_time_str = "10:00" # @param {type:"string"}

# @markdown **2. Filter- und Berechnungsoptionen:**
hide_closed_shops = True # @param {type:"boolean"}
only_slower_window = True # @param {type:"boolean"}

# @markdown **3. POI-Auswahl:**
search_mcdonalds = True # @param {type:"boolean"}

# --- Hier den Pfad zu Ihrer GPX-Datei eintragen ---
gpx_file_path = 'data/2025-06-09_2312053799_Maurice Brocco Vanilla(1).gpx'


def calculate_bearing(p1, p2):
    """
    Berechnet die Peilung (Kurs) zwischen zwei GPX-Punkten.
    """
    lat1 = math.radians(p1.latitude)
    lon1 = math.radians(p1.longitude)
    lat2 = math.radians(p2.latitude)
    lon2 = math.radians(p2.longitude)

    dLon = lon2 - lon1

    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)

    initial_bearing = math.atan2(y, x)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def find_point_at_distance(target_dist, track_points_with_dist):
    """Findet durch Interpolation den genauen Lat/Lon-Punkt für eine gegebene Distanz."""
    if target_dist < 0: return None
    if not track_points_with_dist or target_dist > track_points_with_dist[-1][1]: return None

    for i in range(1, len(track_points_with_dist)):
        p1, d1 = track_points_with_dist[i-1]
        p2, d2 = track_points_with_dist[i]
        
        if d1 <= target_dist <= d2:
            if (d2 - d1) == 0: 
                return (p1.latitude, p1.longitude), 0

            fraction = (target_dist - d1) / (d2 - d1)
            lat = p1.latitude + fraction * (p2.latitude - p1.latitude)
            lon = p1.longitude + fraction * (p2.longitude - p1.longitude)
            
            bearing = calculate_bearing(p1, p2)
            
            return (lat, lon), bearing
            
    return None

def add_time_markers(folium_map, track_points_with_dist, start_datetime_utc, average_speed_kmh):
    """Fügt Linien für volle Stunden senkrecht zur Route hinzu."""
    print("Füge Zeitmarker für volle Stunden hinzu...")
    time_marker_group = folium.FeatureGroup(name="Zeitmarker (Stunden)", show=True)
    
    if not track_points_with_dist: return

    try:
        target_tz = ZoneInfo("Europe/Berlin") # Entspricht MEZ/MESZ (UTC+1/UTC+2)
    except ZoneInfoNotFoundError:
        print("Warnung: Zeitzone 'Europe/Berlin' nicht gefunden. Zeitmarker werden in UTC angezeigt.")
        target_tz = datetime.timezone.utc

    total_distance_m = track_points_with_dist[-1][1]
    speed_ms = (average_speed_kmh * 1000) / 3600
    if speed_ms == 0: return 
    
    total_duration_s = total_distance_m / speed_ms
    
    start_hour = start_datetime_utc.hour
    next_full_hour = (start_hour + 1) % 24
    
    first_hour_marker_time = start_datetime_utc.replace(hour=next_full_hour, minute=0, second=0, microsecond=0)
    if next_full_hour <= start_hour:
        first_hour_marker_time += datetime.timedelta(days=1)

    current_marker_time_utc = first_hour_marker_time
    
    while True:
        elapsed_seconds = (current_marker_time_utc - start_datetime_utc).total_seconds()
        if elapsed_seconds > total_duration_s:
            break

        distance_marker = speed_ms * elapsed_seconds
        
        result = find_point_at_distance(distance_marker, track_points_with_dist)
        if result:
            point_coords, bearing = result
            
            perp_bearing1 = (bearing + 90) % 360
            perp_bearing2 = (bearing - 90 + 360) % 360
            
            line_end1 = geodesic(kilometers=0.5).destination(point_coords, perp_bearing1)
            line_end2 = geodesic(kilometers=0.5).destination(point_coords, perp_bearing2)
            
            folium.PolyLine(
                locations=[(line_end1.latitude, line_end1.longitude), (line_end2.latitude, line_end2.longitude)],
                color='purple', weight=2, opacity=0.7
            ).add_to(time_marker_group)
            
            display_time = current_marker_time_utc.astimezone(target_tz)

            folium.Marker(
                location=[line_end1.latitude, line_end1.longitude],
                icon=folium.DivIcon(html=f"<div style='font-size: 10pt; color: purple; font-weight: bold;'>{display_time.strftime('%H:%M')}</div>")
            ).add_to(time_marker_group)

        current_marker_time_utc += datetime.timedelta(hours=1)
        
    time_marker_group.add_to(folium_map)

def find_closest_point_on_track(poi_coords, track_points_with_dist):
    """Findet den nächstgelegenen Punkt auf der Route zu einem gegebenen POI."""
    min_dist = float('inf')
    closest_point_info = None
    for point, distance_from_start in track_points_with_dist:
        dist = geodesic(poi_coords, (point.latitude, point.longitude)).meters
        if dist < min_dist:
            min_dist = dist
            closest_point_info = (point, distance_from_start)
    return closest_point_info

def fetch_and_add_pois(folium_map, gpx, average_speed_kmh, start_datetime_utc, track_points_with_dist, hide_closed, only_slower, search_mc):
    """Fragt die Overpass API nach POIs entlang der Route ab und prüft die Öffnungszeiten."""
    print("Suche nach Geschäften entlang der Route...")
    
    all_track_points = [p[0] for p in track_points_with_dist]
    CHUNK_SIZE = 150
    processed_poi_ids = set()
    total_pois_found = 0
    poi_group = folium.FeatureGroup(name="Geschäfte", show=True)
    tf = TimezoneFinder()

    num_chunks = (len(all_track_points) + CHUNK_SIZE - 1) // CHUNK_SIZE
    print(f"Lange Route erkannt. Wird in {num_chunks} Abschnitte aufgeteilt.")

    for i in range(0, len(all_track_points), CHUNK_SIZE):
        chunk = all_track_points[i:i + CHUNK_SIZE + 1]
        print(f"Verarbeite Abschnitt {i // CHUNK_SIZE + 1} von {num_chunks}...")

        flat_coords = [str(coord) for point in chunk for coord in (point.latitude, point.longitude)]
        points_str = ",".join(flat_coords)
        
        # Baut die Abfrage dynamisch zusammen
        mcdonalds_query_part = ""
        if search_mc:
            mcdonalds_query_part = f'nwr(around:1000,{points_str})["amenity"="fast_food"]["brand"~"McDonald\'s",i];'

        overpass_query = f"""
        [out:json][timeout:90];(
          nwr(around:1000,{points_str})["amenity"="fuel"];
          nwr(around:1000,{points_str})["shop"="supermarket"];
          nwr(around:1000,{points_str})["shop"="convenience"];
          {mcdonalds_query_part}
        );out center;
        """
        
        overpass_url = "https://overpass-api.de/api/interpreter"
        try:
            response = requests.post(overpass_url, data={'data': overpass_query})
            response.raise_for_status()
            data = response.json()
            
            if not data['elements']: continue

            for element in data['elements']:
                element_id = element['id']
                if element_id in processed_poi_ids: continue
                
                processed_poi_ids.add(element_id)
                
                lat = element.get('lat') or element.get('center', {}).get('lat')
                lon = element.get('lon') or element.get('center', {}).get('lon')
                if not lat or not lon: continue
                
                closest_point_tuple = find_closest_point_on_track((lat, lon), track_points_with_dist)
                if not closest_point_tuple: continue
                
                distance_to_poi_m = closest_point_tuple[1]
                
                speed_avg_ms = (average_speed_kmh * 1000) / 3600
                travel_time_avg_s = distance_to_poi_m / speed_avg_ms if speed_avg_ms > 0 else 0
                eta_avg_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_avg_s)
                
                if only_slower:
                    speed_slower_ms = speed_avg_ms * 0.9
                    travel_time_slower_s = distance_to_poi_m / speed_slower_ms if speed_slower_ms > 0 else 0
                    eta_earliest_utc = eta_avg_utc
                    eta_latest_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_slower_s)
                else: 
                    speed_faster_ms = speed_avg_ms * 1.1 
                    speed_slower_ms = speed_avg_ms * 0.9
                    travel_time_faster_s = distance_to_poi_m / speed_faster_ms if speed_faster_ms > 0 else 0
                    travel_time_slower_s = distance_to_poi_m / speed_slower_ms if speed_slower_ms > 0 else 0
                    eta_earliest_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_faster_s)
                    eta_latest_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_slower_s)
                    
                opening_hours_str = element.get('tags', {}).get('opening_hours')
                status_text = "<i>Öffnungszeiten unbekannt</i>"
                is_open = False
                
                eta_avg_local, eta_earliest_local, eta_latest_local = eta_avg_utc, eta_earliest_utc, eta_latest_utc

                if opening_hours_str:
                    try:
                        tz_str = tf.timezone_at(lng=lon, lat=lat)
                        if tz_str:
                           local_tz = ZoneInfo(tz_str)
                           eta_avg_local = eta_avg_utc.astimezone(local_tz)
                           eta_earliest_local = eta_earliest_utc.astimezone(local_tz)
                           eta_latest_local = eta_latest_utc.astimezone(local_tz)

                           oh = OpeningHours(opening_hours_str)
                           is_open = oh.is_open(eta_avg_local)
                           
                           status_color = 'green' if is_open else 'red'
                           status_word = 'Geöffnet' if is_open else 'Geschlossen'
                           status_text = f"<strong style='color:{status_color};'>{status_word}</strong> bei Ankunft"
                        else:
                           status_text = "<i>Zeitzone unbekannt</i>"
                    except Exception:
                        status_text = "<i>Fehler bei Auswertung der Öffnungszeiten</i>"
                
                if hide_closed and not is_open:
                    continue

                total_pois_found += 1
                poi_type = "Unbekannt"
                poi_name = element.get('tags', {}).get('name', 'Unbenannt')
                icon_color = 'gray'; icon_symbol = 'question'
                tags = element.get('tags', {})
                if tags.get('amenity') == 'fuel': poi_type, icon_color, icon_symbol = 'Tankstelle', 'red', 'tint'
                elif tags.get('shop') == 'supermarket': poi_type, icon_color, icon_symbol = 'Supermarkt', 'blue', 'shopping-cart'
                elif tags.get('shop') == 'convenience': poi_type, icon_color, icon_symbol = 'Kiosk', 'green', 'shopping-basket'
                elif tags.get('amenity') == 'fast_food' and 'mcdonald' in tags.get('brand','').lower():
                    poi_type, icon_color, icon_symbol = "McDonald's", 'orange', 'cutlery'


                popup_html = f"""<b>{poi_type}: {poi_name}</b><br><hr style='margin: 3px 0;'><b>Status:</b> {status_text}<br><b>Ankunft (ca.):</b> {eta_avg_local.strftime('%A, %H:%M')} Uhr<br><small><i>Fenster: {eta_earliest_local.strftime('%H:%M')} - {eta_latest_local.strftime('%H:%M')}</i></small><br><small>Regel: {opening_hours_str or 'n/a'}</small>"""
                folium.Marker(location=[lat, lon], popup=popup_html, icon=folium.Icon(color=icon_color, icon=icon_symbol, prefix='fa')).add_to(poi_group)
        
        except requests.exceptions.RequestException as e:
            print(f"Fehler bei Abschnitt {i // CHUNK_SIZE + 1}: {e}")
        except json.JSONDecodeError:
            print(f"Fehler bei Abschnitt {i // CHUNK_SIZE + 1}: Ungültige Antwort vom Server.")
            
    if total_pois_found > 0:
        print(f"\nInsgesamt {total_pois_found} einzigartige Geschäfte gefunden und zur Karte hinzugefügt.")
        poi_group.add_to(folium_map)
    else:
        print("\nKeine Geschäfte entlang der gesamten Route gefunden.")

def render_gpx_on_map(gpx_file_path, average_speed_kmh, start_date_str, start_time_str, hide_closed, only_slower, search_mc):
    """Liest eine GPX-Datei, rendert die Route und speichert sie als HTML."""
    try:
        naive_datetime = datetime.datetime.strptime(f"{start_date_str} {start_time_str}", "%Y-%m-%d %H:%M")
        start_datetime_local = naive_datetime.astimezone()
        start_datetime_utc = start_datetime_local.astimezone(datetime.timezone.utc)

        with open(gpx_file_path, 'r', encoding='utf-8') as gpx_file:
            gpx = gpxpy.parse(gpx_file)

        points = []
        track_points_with_dist = []
        dist_so_far = 0.0
        for track in gpx.tracks:
            for segment in track.segments:
                for i, point in enumerate(segment.points):
                    if i > 0:
                        dist_so_far += point.distance_2d(segment.points[i-1])
                    track_points_with_dist.append((point, dist_so_far))
                    points.append(tuple([point.latitude, point.longitude]))

        if not points: 
            print("Fehler: Keine Punkte in der GPX-Datei gefunden.")
            return

        map_center = points[0]
        m = folium.Map(location=map_center, zoom_start=13, tiles='OpenStreetMap')
        folium.PolyLine(points, color="#3388ff", weight=5, opacity=0.8, popup=gpx.tracks[0].name, name="GPX Route").add_to(m)

        fetch_and_add_pois(m, gpx, average_speed_kmh, start_datetime_utc, track_points_with_dist, hide_closed, only_slower, search_mc)
        add_time_markers(m, track_points_with_dist, start_datetime_utc, average_speed_kmh)
        
        folium.LayerControl().add_to(m)
        sw = min(points, key=lambda x: x[0])[0], min(points, key=lambda x: x[1])[1]
        ne = max(points, key=lambda x: x[0])[0], max(points, key=lambda x: x[1])[1]
        m.fit_bounds([sw, ne])

        base_gpx_name = os.path.splitext(os.path.basename(gpx_file_path))[0]
        output_filename = f"{base_gpx_name}_routenkarte.html"
        m.save(output_filename)
        
        print(f"\nErfolg! Route '{gpx.tracks[0].name or 'Unbenannte Route'}' wird dargestellt.")
        print(f"✅ Karte erfolgreich als '{output_filename}' im selben Verzeichnis wie Ihr Notebook gespeichert.")
        return m

    except FileNotFoundError:
        print(f"Fehler: Die Datei wurde nicht unter '{gpx_file_path}' gefunden.")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# Führt die Funktion aus und zeigt die Karte an
gpx_map = render_gpx_on_map(gpx_file_path, average_speed_kmh, start_date, start_time_str, hide_closed_shops, only_slower_window, search_mcdonalds)
gpx_map


Suche nach Geschäften entlang der Route...
Lange Route erkannt. Wird in 61 Abschnitte aufgeteilt.
Verarbeite Abschnitt 1 von 61...
Verarbeite Abschnitt 2 von 61...
Verarbeite Abschnitt 3 von 61...
Verarbeite Abschnitt 4 von 61...
Verarbeite Abschnitt 5 von 61...
Verarbeite Abschnitt 6 von 61...
Verarbeite Abschnitt 7 von 61...
Verarbeite Abschnitt 8 von 61...
Verarbeite Abschnitt 9 von 61...
Verarbeite Abschnitt 10 von 61...
Verarbeite Abschnitt 11 von 61...
Verarbeite Abschnitt 12 von 61...
Verarbeite Abschnitt 13 von 61...
Verarbeite Abschnitt 14 von 61...
Verarbeite Abschnitt 15 von 61...
Verarbeite Abschnitt 16 von 61...
Verarbeite Abschnitt 17 von 61...
Verarbeite Abschnitt 18 von 61...
Verarbeite Abschnitt 19 von 61...
Verarbeite Abschnitt 20 von 61...
Verarbeite Abschnitt 21 von 61...
Verarbeite Abschnitt 22 von 61...
Verarbeite Abschnitt 23 von 61...
Verarbeite Abschnitt 24 von 61...
Verarbeite Abschnitt 25 von 61...
Verarbeite Abschnitt 26 von 61...
Verarbeite Abschnitt 27 von